In [17]:
#!pip install xgboost

In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# # Exclude boolean columns (location_A, location_B, location_C)
# columns_to_scale = x_train.columns.difference(['location_A', 'location_B', 'location_C'])

# scaler = MinMaxScaler()
# x_train[columns_to_scale] = scaler.fit_transform(x_train[columns_to_scale])
# x_test[columns_to_scale] = scaler.transform(x_test[columns_to_scale])


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [19]:
x_train = pd.read_csv('cleaned_and_combined_data/x_train_combined.csv')
y_train = pd.read_csv('cleaned_and_combined_data/y_train_combined.csv')
x_test = pd.read_csv('cleaned_and_combined_data/x_test_combined.csv')

# Determine the split index
split_index = int(0.875 * len(x_train))

# Split the data into training and validation sets
x_train_df = x_train[:split_index]
y_train_df = y_train[:split_index]
x_valid_df = x_train[split_index:]
y_valid_df = y_train[split_index:]



In [20]:
from sklearn.preprocessing import MinMaxScaler

# Exclude boolean columns (location_A, location_B, location_C)
# columns_to_scale = x_train.columns.difference(['location_A', 'location_B', 'location_C'])

# scaler = MinMaxScaler()
# x_train[columns_to_scale] = scaler.fit_transform(x_train[columns_to_scale])
# x_test[columns_to_scale] = scaler.transform(x_test[columns_to_scale])


In [21]:


dtrain = xgb.DMatrix(x_train_df,label = y_train_df['pv_measurement'])
dval = xgb.DMatrix(x_valid_df,label = y_valid_df['pv_measurement'])
test = xgb.DMatrix(x_test)


In [22]:
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'booster': 'gbtree',
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.1,
    'lambda': 1,

}
#  Train the model
num_boost_round = 1000  # Maximum number of boosting rounds
early_stopping_rounds = 50  # Stop if validation score doesn't improve for 50 rounds

evals = [(dtrain, 'train'), (dval, 'eval')]
progress = {}


In [23]:
bst = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=evals,
    early_stopping_rounds=early_stopping_rounds,
    evals_result=progress
)

print("Best MAE: {:.2f} with {} rounds".format(
         bst.best_score,
         bst.best_iteration+1))


[0]	train-mae:452.58473	eval-mae:378.20850
[1]	train-mae:431.71483	eval-mae:361.54254
[2]	train-mae:412.10626	eval-mae:345.76168
[3]	train-mae:393.72026	eval-mae:330.92285
[4]	train-mae:376.27797	eval-mae:316.90233
[5]	train-mae:359.88827	eval-mae:303.74271
[6]	train-mae:344.43929	eval-mae:291.23351
[7]	train-mae:330.03668	eval-mae:279.54061
[8]	train-mae:316.31920	eval-mae:268.41997
[9]	train-mae:303.54260	eval-mae:258.03149
[10]	train-mae:291.54325	eval-mae:248.16702
[11]	train-mae:280.21693	eval-mae:239.05823
[12]	train-mae:269.66630	eval-mae:230.08814
[13]	train-mae:259.68628	eval-mae:221.61960
[14]	train-mae:250.30667	eval-mae:213.73687
[15]	train-mae:241.43505	eval-mae:206.17293
[16]	train-mae:233.18643	eval-mae:199.05066
[17]	train-mae:225.39908	eval-mae:192.29870
[18]	train-mae:218.17008	eval-mae:185.98613
[19]	train-mae:211.29225	eval-mae:179.81635
[20]	train-mae:204.82265	eval-mae:174.02157
[21]	train-mae:198.77002	eval-mae:168.68400
[22]	train-mae:193.05248	eval-mae:163.5840

In [24]:
predictions = bst.predict(test)
predictions

array([ 2.7156568,  2.7156568,  2.6367888, ..., -2.3124921, -3.5591257,
       -4.442778 ], dtype=float32)

In [25]:
sample_submission = pd.read_csv('sample_submission.csv')

# Convert the numpy array to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['prediction'])

# Join the 'id' column from sample_submission with the predictions
sample_submission['prediction'] = predictions_df['prediction']

# Save to CSV
sample_submission.to_csv('my_first_submission.csv', index=False)